In [1]:
import numpy as np
import matplotlib

In [2]:
matplotlib.use("Qt5Agg")
from PyQt5 import QtCore
from PyQt5.QtWidgets import QApplication, QMainWindow, QMenu, QVBoxLayout, QSizePolicy, QMessageBox, QWidget

In [3]:
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

In [4]:
import os.path
import dolfyn.adv.api as avm
import dolfyn.adv.turbulence as turb

In [5]:
fname = '/Users/lillie/turbulence_data/raw_data/TTM_NREL03_May2015'

In [6]:
%%time
if os.path.isfile(fname + '.h5'):
    dat_raw = avm.load(fname + '.h5')
else:
    dat_raw = avm.read_nortek(fname + '.VEC')
    dat_raw.save(fname + '.h5')   

CPU times: user 1.43 s, sys: 190 ms, total: 1.62 s
Wall time: 1.62 s


In [25]:
def onpick(event):
    ind = event.ind
    print('onpick scatter:', ind, np.take(dat_raw.mpltime, ind), np.take(dat_raw.u, ind))

In [7]:
# A class that will downsample the data and recompute when zoomed.
class DataDisplayDownsampler(object):
    def __init__(self, xdata, ydata):
        self.origYData = ydata
        self.origXData = xdata
        self.ratio = 5
        self.delta = xdata[-1] - xdata[0]

    def downsample(self, xstart, xend):
        # Very simple downsampling that takes the points within the range
        # and picks every Nth point
        mask = (self.origXData > xstart) & (self.origXData < xend)
        xdata = self.origXData[mask]
        xdata = xdata[::self.ratio]

        ydata = self.origYData[mask]
        ydata = ydata[::self.ratio]

        return xdata, ydata
    
    def update(self, ax):
        # Update the line
        lims = ax.viewLim
        if np.abs(lims.width - self.delta) > 1e-8:
            self.delta = lims.width
            xstart, xend = lims.intervalx
            self.line.set_data(*self.downsample(xstart, xend))
            ax.figure.canvas.draw_idle()

In [8]:
d = DataDisplayDownsampler(dat_raw.mpltime, dat_raw.u)

In [13]:
%matplotlib notebook

fig, ax = plt.subplots()
plt.switch_backend
# Hook up the line
d.line, = ax.plot(dat_raw.mpltime, dat_raw.u, 'o-')
ax.set_autoscale_on(False)  # Otherwise, infinite loop

# Connect for changing the view limits
ax.callbacks.connect('xlim_changed', d.update)

plt.show()

In [9]:
t_range_start = .0082 + 7.3572944e5
t_range_end = .005 + 7.357313e5
t_range = [t_range_start, t_range_end]
t_range

[735729.4482, 735731.305]

In [19]:
print(os.environ.get('QT_API'))

None
